# This version of program is specific for IGH[VDJ]

- [ ] Generalize for Ig light chain loci and TCR alpha and beta chains

- [X] Account for inexact matches between found genes and IMGT records

In [47]:
from Bio.Seq import Seq
from Bio import SeqIO
from os.path import basename
from re import finditer
from re import match
from argparse import ArgumentParser
from collections import OrderedDict

In [2]:
locus_file = './Loci/IGH_locus.fasta'

In [3]:
dest_dir = './IMGT_human/IMGT/'

In [4]:
vgene_file = dest_dir+'IGHV.fasta'
dgene_file = dest_dir+'IGHD.fasta'
jgene_file = dest_dir+'IGHJ.fasta'

_The following cell is for the command line execution of this code, and is not needed in the notebook version_

In [5]:
#parser = ArgumentParser(description='Find VDJ genes in long reads')
#parser.add_argument('-f', dest='locus_file', help='Input fasta file')
#parser.add_argument('-v', dest='vgene_file', help='V gene reference')
#parser.add_argument('-d', dest='dgene_file', help='D gene reference')
#parser.add_argument('-j', dest='jgene_file', help='J gene reference')
#args = parser.parse_args()
#locus_file = args.locus_file
#vgene_file = args.vgene_file
#dgene_file = args.dgene_file
#jgene_file = args.jgene_file

### Setting up comparison logic for genes
##### Called by the main program
- compare_dict():
    - returns dictionary of matches for given gene in seq_dict
    - keys are the allele argument of seq, value is match percent

##### Intended to be private to the compare_dict() function
- compare():
    - returns indicator based on if the first argument is longer or shorter than the second
    - designed to support Python 'str1 in str2' search method
- compare_substring():
    - returns dictionary of partial matches found for given gene,seq argument
    - designed to continue search for a match in seq_dict by looking up all possible substrings of gene
- compare_partials():
    - returns dictionary of all partial matches for gene across seq_dict
    - designed to behave like compare_dict() but perform more intricate search for match

In [6]:
def compare( gene, seq ):
    if len(gene) > len(seq):
        return 1
    elif len(gene) < len(seq):
        return -1
    return 0

In [7]:
def compare_substring( gene, seq, allele ):
    partials = {}
    
    for i in range(len(gene)):
        for j in range(1, len(gene)+1):
            partial = gene[i:-j:1]
            
            if partial in seq:
                match = round( len(partial)/len(seq), 3 )
                
                if (allele not in partials) and (match > 0.1):
                    partials[allele] = match
                elif allele in partials and partials[allele] < match:
                    partials[allele] = match
                    
                break
    return partials

In [8]:
def compare_partials( gene, seq_dict ):
    matches = {}
    
    for allele, seq in seq_dict.items():
        partials = compare_substring(gene, seq, allele)
        
        if partials:
            matches.update(partials)
            
    return matches

In [9]:
def compare_dict( gene, seq_dict ):
    matches = {}
    
    for allele, seq in seq_dict.items():
        comp = compare(gene, seq)
        match_perc = round(len(gene)/len(seq), 3)
        
        if comp > 0:
            if seq in gene:
                matches[allele] = match_perc
        elif comp < 0:
            if gene in seq:
                matches[allele] = match_perc
        else:
            if gene == seq:
                matches[allele] = match_perc
                
    if matches:
        return matches
    else:
        return compare_partials( gene, seq_dict )

### Output of Original Script
Comparing the total number of genes identified in given .fasta by
- The number of genes marked 'found'
- The number of genes marked 'Not found'

**********************************************************************************************************************
#### Per the IGHV_found.fasta file, of the 50 genes returned:
 - 47 were found in the vseq_dict
 - 3 were identified as not in the vseq_dict

However, there are 195 vgenes identified by the script, so there are 145 identified but not returned.

#### Per the IGHD_found.fasta file, of the 21 genes returned:
- all 21 genes were found in the dseq_dict

However, there are 31 dgenes identified by the script, so 10 are identified but not returned.

#### Per the IGHJ_found.fasta file, of the 6 genes returned:
- 1 of these were found in the jseq_dict
- 5 were identified as not in the jseq_dict

However, there are 186 jgenes identified by the script, so 180 are identified but not returned.
**********************************************************************************************************************

In [13]:
# Collection of all the genes identified by the program
#all_returns

OrderedDict([('caggtgcagctggtgcagtctggccatgaggtgaagcagcctggggcctcagtgaaggtctcctgcaaggcttctggttacagtttcaccacctatggtatgaattgggtgccacaggcccctggacaagggcttgagtggatgggatggttcaacacctacactgggaacccaacatatgcccagggcttcacaggacggtttgtcttctccatggacacctctgccagcacagcatacctgcagatcagcagcctaaaggctgaggacatggccatgtattactgtgcgagata',
              'IGHV7-81*01 ORF'),
             ('attaaaaagttaggaaacaacaggtgctggagaggatgtggagaaataggaacatttttacactgttggtgggactgtaaactagttcaaccattatggaagtcagtgtggcaattcctcagggatctggaactggaaataccatttgacccagccatcccattactgggtatatacccaaaggactataaatcatgctgctataaagacacatgcacacgtatgtttattgcggcattattcacaatagcaaagacttggaaccaacccaaatgt',
              'Not in V ref db'),
             ('cccacttaaacccagggctctcctccacagtgagtctccttcaccacccagctgggatctcagtgctttcttttctgtcctcctccaggatggggtcaaccgtcatcctttccctcgtcctggctgttctccaaggtcagtcctgcccagggtttgaggtcacagagaagaatgggcagaagggagcccctgatgcaaattttgtgtctcccacacaggtgtctttgccgaggtgcagctgttgcagtctgc',
              'Not in V ref db'),
             ('tgagg

# Parse the V, D and J gene files
## Notes
- Partial in 5' means may be missing start of 5' region
- Partial in 3' means may be missing start of 3' region
- Max IGHV gene = 305 nt

### V genes

In [10]:
vseq_dict   = {} # Dictionary of V gene sequences (key = allele, value = sequence)
vtype_dict  = {} # Dictionary of V gene types (key = allele, value = gene type)

In [11]:
if vgene_file:
    vgene_out = basename(vgene_file).replace('.fasta', '_found.fasta')
    vout = open(vgene_out, 'w')
    for vnt in SeqIO.parse(vgene_file, "fasta"):
        vnts    = str(vnt.seq).lower()
        p = vnt.description.split('|')
        vallele = p[1]
        vtype   = p[3]
        vseq_dict[vallele]  = vnts
        vtype_dict[vallele] = vtype

### _D genes_

In [12]:
dseq_dict   = {} # Dictionary of D gene sequences (key = allele, value = sequence)
dtype_dict  = {} # Dictionary of D gene types (key = allele, value = gene type)

In [13]:
if dgene_file:
    dgene_out = basename(dgene_file).replace('.fasta', '_found.fasta')
    dout = open(dgene_out, 'w')
    for dnt in SeqIO.parse(dgene_file, "fasta"):
        dnts    = str(dnt.seq).lower()
        p = dnt.description.split('|')
        dallele = p[1]
        dtype   = p[3]
        dseq_dict[dallele]  = dnts
        dtype_dict[dallele] = dtype

### _J genes_

In [14]:
jseq_dict   = {} # Dictionary of J gene sequences (key = allele, value = sequence)
jtype_dict  = {} # Dictionary of J gene types (key = allele, value = gene type)

In [15]:
if jgene_file:
    jgene_out = basename(jgene_file).replace('.fasta', '_found.fasta')
    jout = open(jgene_out, 'w')
    for jnt in SeqIO.parse(jgene_file, "fasta"):
        jnts    = str(jnt.seq).lower()
        p = jnt.description.split('|')
        jallele = p[1]
        jtype   = p[3]
        jseq_dict[jallele]  = jnts
        jtype_dict[jallele] = jtype

### Read fasta sequence

In [17]:
# Predefined consensus to compare input to
# hept_cons = up_hept_cons = down_hept_cons = 'cacagtg'
# non_cons = down_non_cons = 'acaaaaacc'
# up_non_cons_d = 'tgtttttgt'
# up_non_cons_j = 'ggtttttgt'

In [68]:
all_returns = {}
vgenes = []
printed = 0

# Read fasta sequence
for nt in SeqIO.parse(locus_file, "fasta"):
    nts = str(nt.seq).lower()

    # Get nt sequences in three reading frames
    nt_len = len(nt.seq)
    nt_frame = ['','','']
    nt_frame[0] = nt[0 : nt_len - (nt_len + 0)%3]
    nt_frame[1] = nt[1 : nt_len - (nt_len + 2)%3]
    nt_frame[2] = nt[2 : nt_len - (nt_len + 1)%3]

    # Get aa translations in three reading frames
    aa_frame = ['','','']
    aa_frame[0] = str(nt_frame[0].seq.translate())
    aa_frame[1] = str(nt_frame[1].seq.translate())
    aa_frame[2] = str(nt_frame[2].seq.translate())

    # Uncomment to skip search for V and J genes
    #aa_frame[0] = ''; aa_frame[1] = ''; aa_frame[2] = ''

    # -------------------- SEARCH FOR V GENES --------------------

    # Find matches for V-genes (1st conserved Cys to 2nd conserved Cys)
    # Max number of residues between Cys1-Trp       = 17
    # Max number of residues between Tpr-[IVLFCMA]  = 47
    # Max number of residues between [IVLFCMA]-Cys2 = 14
    # Allow for missing residues - 9 in Cys1-Trp, 9 in Trp-[IVLFCMA], 2 in [IVLFCMA]-Cys2


    for frame,offset in zip(aa_frame,[0,1,2]):
        for mcons in finditer('C[A-Z]{8,17}W[A-Z]{38,47}[IVLFCMA][A-Z]{12,14}C', frame):
            sta_aa = mcons.span()[0]         # Starting aa in frame
            end_aa = mcons.span()[1]         # End aa in frame
            sta_nt = sta_aa*3 + offset       # Start nt in original seq
            end_nt = end_aa*3 + offset       # End nt in original sequence
            
            # Search for 3' flanking sequence
            found     = False
            heptamer  = ''
            spacer    = ''
            nonamer   = ''
            post_cys2 = 0
            flank = nts[end_nt:end_nt+60]
            for i in range(0,20):
                heptamer  = flank[i:i+7]
                spacer  = flank[i+7:i+7+23]
                nonamer = flank[i+7+23:i+7+23+9]

                # heptamer consensus = cacagtg
                heptamer_score = 0
                if heptamer[3] == 'a': heptamer_score += 1
                if heptamer[4] == 'g': heptamer_score += 1
                if heptamer[5] == 't': heptamer_score += 1
                if heptamer[6] == 'g': heptamer_score += 1
                
                # nonamer consensus = acaaaaacc
                nonamer_score = 0
                if nonamer[0] == 'a': nonamer_score += 1
                if nonamer[1] == 'c': nonamer_score += 1
                if nonamer[2] == 'a': nonamer_score += 1
                if nonamer[3] == 'a': nonamer_score += 1
                if nonamer[4] == 'a': nonamer_score += 1
                if nonamer[5] == 'a': nonamer_score += 1
                if nonamer[6] == 'a': nonamer_score += 1
                if nonamer[7] == 'c': nonamer_score += 1
                if nonamer[8] == 'c': nonamer_score += 1

                # Setting nonamer_score lower than 6 leads to more false positives and breaks results
                if heptamer[0:3] == 'cac' and heptamer_score >= 2 and nonamer_score >= 6:
                    end_nt += i
                    post_cys2 = i
                    found = True
                    break

            # Set start of gene to 63 bases before first conserved Cys in IGH
            # (Use 66 bases for IGK and IGL)
            sta_nt  -= 63
            gene     = nts[sta_nt:end_nt]
            aa       = frame[sta_aa-21:end_aa+int(post_cys2/3)]
            gene_len = end_nt - sta_nt

            vgenes.append(gene)
            
            # Test V gene for stop codons
            if '*' in aa:
                stop_codon = '(Contains stop codon)'
            else:
                stop_codon = ''

            # Look for exact matches between discovered gene and known alleles
            # Allow for the possibility that multiple alleles have same sequence
            # Default assumption is that gene is not in database
            allele = 'Not in V ref db'
            for vallele, vseq in vseq_dict.items():
                if gene == vseq:
                    if allele == 'Not in V ref db':
                        allele = vallele + ' ' + vtype_dict[vallele]
                    else:
                        allele = allele + ', ' + vallele + ' ' + vtype_dict[vallele]
            
            if found:
                printed += 1
                print(f">{allele} {gene_len} nts: {sta_nt} - {end_nt} {stop_codon}\n")
                print(gene)
                print("\n")
                last_v_nt = end_nt # Keep track of where last V gene found
                
            all_returns[gene] = allele

>IGHV7-81*01 ORF 296 nts: 4968 - 5264 

caggtgcagctggtgcagtctggccatgaggtgaagcagcctggggcctcagtgaaggtctcctgcaaggcttctggttacagtttcaccacctatggtatgaattgggtgccacaggcccctggacaagggcttgagtggatgggatggttcaacacctacactgggaacccaacatatgcccagggcttcacaggacggtttgtcttctccatggacacctctgccagcacagcatacctgcagatcagcagcctaaaggctgaggacatggccatgtattactgtgcgagata


>IGHV3-73*02 F 302 nts: 76851 - 77153 

gaggtgcagctggtggagtccgggggaggcttggtccagcctggggggtccctgaaactctcctgtgcagcctctgggttcaccttcagtggctctgctatgcactgggtccgccaggcttccgggaaagggctggagtgggttggccgtattagaagcaaagctaacagttacgcgacagcatatgctgcgtcggtgaaaggcaggttcaccatctccagagatgattcaaagaacacggcgtatctgcaaatgaacagcctgaaaaccgaggacacggccgtgtattactgtactagaca


>IGHV3-72*01 F 302 nts: 88851 - 89153 

gaggtgcagctggtggagtctgggggaggcttggtccagcctggagggtccctgagactctcctgtgcagcctctggattcaccttcagtgaccactacatggactgggtccgccaggctccagggaaggggctggagtgggttggccgtactagaaacaaagctaacagttacaccacagaatacgccgcgtctgtgaaaggcagattcaccatctcaagagatgattcaaagaactcactgtatctgcaaatgaacagcctgaaaaccgaggac

In [69]:
all_returns

{'aaaatgtataggtttaaggtgccccacttgacgttttgatgttgtattagtccattctcacactgctataaaaaatgcctgagactgagtaatttgaaaaaaaatatgtttaattggctcacggttctgcaggctgtacaggaagtgcagtggcttctccttctggggagagtcaggaaagttaggatcatggcagaagacaaaggggagcaggtgtgtcacatggccagagcggaagccacaggaagcggaccggggtgtggggggactgcacactgt': 'Not in V ref db',
 'aaaccaagccatacagagataggagtggaagggacatggtgagaagtgaccagaagacaagactgcgagccttctgttatgcccggacatggccaccagagggctccttggtctagcggtaacaccagcttctgggaagatgcccgttgccaagcagaccgtggtctagcggtagcgtcagtgtcaaggaaaaacacccgctacttagcagaccaggacagggagtctccctttccccaggggagtttagagaagactctcctcctccacctcttgtgt': 'Not in V ref db',
 'aagcactgctatgtgtgaagaaacaccacagcatactctacactgtcactcatgatatggaattgttgcctcctcagcaatgtaactgagcccttaatatgtggctgggatgacaaagcaatgccagcctgcagagtggaacctgttcttccttctgggattagccaaaggccggtggcactcatggctgaccctactttccttacaactggacgaagcgaagcaactggatatgaacttctatcccaggatgtaactgtccaaagtgccttctctttatgt': 'Not in V ref db',
 'aagggcaggccgggtccttgtggagagcacatttagtgggagggacatgatttcccttcaaagtgcccattctggacgcttcccgt

In [73]:
len(all_returns.keys())

195

In [74]:
not_found = [ gene for gene,allele in all_returns.items() if allele == 'Not in V ref db']
len(not_found)

148

In [77]:
partials_v = {}

for gene in not_found:
    partials_v[gene] = compare_partials( gene,vseq_dict )

partials_v

{'aaaatgtataggtttaaggtgccccacttgacgttttgatgttgtattagtccattctcacactgctataaaaaatgcctgagactgagtaatttgaaaaaaaatatgtttaattggctcacggttctgcaggctgtacaggaagtgcagtggcttctccttctggggagagtcaggaaagttaggatcatggcagaagacaaaggggagcaggtgtgtcacatggccagagcggaagccacaggaagcggaccggggtgtggggggactgcacactgt': {},
 'aaaccaagccatacagagataggagtggaagggacatggtgagaagtgaccagaagacaagactgcgagccttctgttatgcccggacatggccaccagagggctccttggtctagcggtaacaccagcttctgggaagatgcccgttgccaagcagaccgtggtctagcggtagcgtcagtgtcaaggaaaaacacccgctacttagcagaccaggacagggagtctccctttccccaggggagtttagagaagactctcctcctccacctcttgtgt': {},
 'aagcactgctatgtgtgaagaaacaccacagcatactctacactgtcactcatgatatggaattgttgcctcctcagcaatgtaactgagcccttaatatgtggctgggatgacaaagcaatgccagcctgcagagtggaacctgttcttccttctgggattagccaaaggccggtggcactcatggctgaccctactttccttacaactggacgaagcgaagcaactggatatgaacttctatcccaggatgtaactgtccaaagtgccttctctttatgt': {},
 'aagggcaggccgggtccttgtggagagcacatttagtgggagggacatgatttcccttcaaagtgcccattctggacgcttcccgttccatgctggacgcttcctcttccacgctggatgcttcctgttcc

In [98]:
top_matches_v = []

for gene,matches in partials.items():
    for allele,perc in matches.items():
        if perc >= 0.75:
            top_matches_v.append( (allele,perc,gene) )

top_matches_v

[('IGHV5-78*01',
  0.976,
  'gaggtgcagctgttgcagtctgcagcagaggtgaaaagacccggggagtctctgaggatctcctgtaagacttctggatacagctttaccagctactggatccactgggtgcgccagatgcccgggaaagaactggagtggatggggagcatctatcctgggaactctgataccagatacagcccatccttccaaggccacgtcaccatctcagccgacagctccagcagcaccgcctacctgcagtggagcagcctgaaggcctcggacgccgccatgtattattgt'),
 ('IGHV3-47*01',
  1.0,
  'gaggatcagctggtggagtctgggggaggcttggtacagcctggggggtccctgcgaccctcctgtgcagcctctggattcgccttcagtagctatgctctgcactgggttcgccgggctccagggaagggtctggagtgggtatcagctattggtactggtggtgatacatactatgcagactccgtgatgggccgattcaccatctccagagacaacgccaagaagtccttgtatcttcatatgaacagcctgatagctgaggacatggctgtgtattattgtgcaagaga'),
 ('IGHV3-16*01',
  0.939,
  'gaggtgcagctggtggagtctgggggaggcttggtacagcctggggggtccctgagactctcctgtgcagcctctggattcaccttcagtaacagtgacatgaactgggcccgcaaggctccaggaaaggggctggagtgggtatcgggtgttagttggaatggcagtaggacgcactatgtggactccgtgaagcgccgattcatcatctccagagacaattccaggaactccctgtatctgcaaaagaacagacggagagccgaggacatggctgtgtattactgt'),
 ('IGHV3-16*02',
  0.97,
  'gaggtg

##### D genes

In [78]:
# -------------------- SEARCH FOR D GENES --------------------
dgenes = []
printed = 0

# Read fasta sequence
for nt in SeqIO.parse(locus_file, "fasta"):
    nts = str(nt.seq).lower()

    # Get nt sequences in three reading frames
    nt_len = len(nt.seq)
    nt_frame = ['','','']
    nt_frame[0] = nt[0 : nt_len - (nt_len + 0)%3]
    nt_frame[1] = nt[1 : nt_len - (nt_len + 2)%3]
    nt_frame[2] = nt[2 : nt_len - (nt_len + 1)%3]

    # Get aa translations in three reading frames
    aa_frame = ['','','']
    aa_frame[0] = str(nt_frame[0].seq.translate())
    aa_frame[1] = str(nt_frame[1].seq.translate())
    aa_frame[2] = str(nt_frame[2].seq.translate())

    # Uncomment to skip search for V and J genes
    #aa_frame[0] = ''; aa_frame[1] = ''; aa_frame[2] = ''
    
    
    #for dseq in re.finditer('cac[acgt]gtg[acgt]{10,32}cac[acgt]gtg', nts):
    #w/ hept>=2, non>=4 16 true pos, 0 false pos

    #for dseq in re.finditer('[acgt]ac[acgt]gtg[acgt]{10,32}cac[acgt]gtg', nts):
    # w/ hept>=2, non>=4 22 true pos, 2 false pos

    for dseq in finditer('[acgt]ac[acgt]gtg[acgt]{10,37}cac[acgt]gtg', nts):
        sta_nt = dseq.span()[0]
        end_nt = dseq.span()[1]
        gene = nts[sta_nt+7:end_nt-7]
        spacer = nts[end_nt:end_nt+12]
        gene_len = end_nt - sta_nt - 14
        
        # Temporary collection of genes identified by script
        dgenes.append(gene)
        
        # upstream heptamer consensus = cactgtg
        upstream_heptamer = nts[sta_nt:sta_nt+7]
        upstream_heptamer_score = 0
        if upstream_heptamer[0] == 'c': upstream_heptamer_score += 1
        if upstream_heptamer[1] == 'a': upstream_heptamer_score += 1
        if upstream_heptamer[2] == 'c': upstream_heptamer_score += 1
        if upstream_heptamer[3] == 't': upstream_heptamer_score += 1

        # upstream nonamer consensus = tgtttttgt
        upstream_nonamer = nts[sta_nt-12-9:sta_nt-12]
        upstream_nonamer_score = 0
        if upstream_nonamer[0] == 't': upstream_nonamer_score += 1
        if upstream_nonamer[1] == 'g': upstream_nonamer_score += 1
        if upstream_nonamer[2] == 't': upstream_nonamer_score += 1
        if upstream_nonamer[3] == 't': upstream_nonamer_score += 1
        if upstream_nonamer[4] == 't': upstream_nonamer_score += 1
        if upstream_nonamer[5] == 't': upstream_nonamer_score += 1
        if upstream_nonamer[6] == 't': upstream_nonamer_score += 1
        if upstream_nonamer[7] == 'g': upstream_nonamer_score += 1
        if upstream_nonamer[8] == 'g': upstream_nonamer_score += 1 # RSS ERROR

        # downstream heptamer consensus = cacagtg
        downstream_heptamer = nts[end_nt-7:end_nt]
        downstream_heptamer_score = 0
        if downstream_heptamer[3] == 'a': downstream_heptamer_score += 1
        if downstream_heptamer[4] == 'g': downstream_heptamer_score += 1
        if downstream_heptamer[5] == 't': downstream_heptamer_score += 1
        if downstream_heptamer[6] == 'g': downstream_heptamer_score += 1

        # downstream nonamer consensus = acaaaaacc
        downstream_nonamer = nts[end_nt+12:end_nt+12+9]
        downstream_nonamer_score = 0
        if downstream_nonamer[0] == 'a': downstream_nonamer_score += 1
        if downstream_nonamer[1] == 'c': downstream_nonamer_score += 1
        if downstream_nonamer[2] == 'a': downstream_nonamer_score += 1
        if downstream_nonamer[3] == 'a': downstream_nonamer_score += 1
        if downstream_nonamer[4] == 'a': downstream_nonamer_score += 1
        if downstream_nonamer[5] == 'a': downstream_nonamer_score += 1
        if downstream_nonamer[6] == 'a': downstream_nonamer_score += 1
        if downstream_nonamer[7] == 'c': downstream_nonamer_score += 1
        if downstream_nonamer[8] == 'c': downstream_nonamer_score += 1


        # Look for exact matches between discovered gene and known alleles
        # Allow for the possibility that multiple alleles have same sequence
        # Default assumption is that gene is not in database

        allele = 'Not in D ref db'
        for dallele, dseq in dseq_dict.items():
            if gene == dseq:
                if allele == 'Not in D ref db':
                    allele = dallele + ' ' + dtype_dict[dallele]
                else:
                    allele = allele + ', ' + dallele + ' ' + dtype_dict[dallele]

        all_returns[gene] = allele
        
        # Flag D genes that start before last V gene
        if sta_nt < last_v_nt:
            continue
            allele += ' / located in V gene region'

        if (upstream_heptamer_score >= 2 and upstream_nonamer_score >= 4 and 
            upstream_heptamer_score >= 2 and upstream_nonamer_score >= 4):
            
            printed_d += 1
            print(f">{allele} {gene_len} nts: {sta_nt} - {end_nt}\n")
            print(gene)
            print("\n")

>IGHD1-1*01 F 17 nts: 960319 - 960350

ggtacaactggaacgac


>IGHD2-2*01 F 31 nts: 962981 - 963026

aggatattgtagtagtaccagctgctatgcc


>IGHD3-3*01 F 31 nts: 965448 - 965493

gtattacgatttttggagtggttattatacc


>IGHD4-11*01 ORF, IGHD4-4*01 F 16 nts: 966600 - 966630

tgactacagtaactac


>IGHD5-18*01 F, IGHD5-5*01 F 20 nts: 967561 - 967595

gtggatacagctatggttac


>IGHD6-6*01 F 18 nts: 969410 - 969442

gagtatagcagctcgtcc


>IGHD1-7*01 F 17 nts: 969914 - 969945

ggtataactggaactac


>IGHD2-8*01 F 31 nts: 972596 - 972641

aggatattgtactaatggtgtatgctatacc


>IGHD3-9*01 F 31 nts: 975126 - 975171

gtattacgatattttgactggttattataac


>IGHD3-10*01 F 31 nts: 975310 - 975355

gtattactatggttcggggagttattataac


>IGHD5-12*01 F 23 nts: 977166 - 977203

gtggatatagtggctacgattac


>IGHD6-13*01 F 21 nts: 978675 - 978710

gggtatagcagcagctggtac


>IGHD2-15*01 F 31 nts: 981850 - 981895

aggatattgtagtggtggtagctgctactcc


>IGHD4-17*01 F 16 nts: 985314 - 985344

tgactacggtgactac


>IGHD5-18*01 F, IGHD5-5*01 F 20 nts: 9862

In [79]:
not_found = [ gene for gene,allele in all_returns.items() if allele == 'Not in D ref db']
len(not_found)

9

In [80]:
partials_d = {}

for gene in not_found:
    partials_d[gene] = compare_partials( gene,dseq_dict )

partials_d

{'aggacactaga': {'IGHD1-1*01': 0.176,
  'IGHD1-14*01': 0.176,
  'IGHD1-20*01': 0.176,
  'IGHD1-26*01': 0.2,
  'IGHD1-7*01': 0.235,
  'IGHD1/OR15-1a*01': 0.176,
  'IGHD1/OR15-1b*01': 0.176,
  'IGHD2-15*01': 0.129,
  'IGHD2-2*01': 0.129,
  'IGHD2-2*02': 0.129,
  'IGHD2-21*01': 0.107,
  'IGHD2-21*02': 0.107,
  'IGHD2-8*01': 0.129,
  'IGHD2-8*02': 0.129,
  'IGHD2/OR15-2a*01': 0.129,
  'IGHD2/OR15-2b*01': 0.129,
  'IGHD3-10*01': 0.129,
  'IGHD3-10*02': 0.133,
  'IGHD3-22*01': 0.129,
  'IGHD3/OR15-3a*01': 0.129,
  'IGHD3/OR15-3b*01': 0.129,
  'IGHD4-11*01': 0.25,
  'IGHD4-17*01': 0.25,
  'IGHD4-23*01': 0.211,
  'IGHD4-4*01': 0.25,
  'IGHD4/OR15-4a*01': 0.211,
  'IGHD4/OR15-4b*01': 0.211,
  'IGHD5-12*01': 0.13,
  'IGHD5-18*01': 0.15,
  'IGHD5-24*01': 0.15,
  'IGHD5-5*01': 0.15,
  'IGHD5/OR15-5a*01': 0.13,
  'IGHD5/OR15-5b*01': 0.13,
  'IGHD6-13*01': 0.143,
  'IGHD6-19*01': 0.143,
  'IGHD6-25*01': 0.167,
  'IGHD6-6*01': 0.167,
  'IGHD7-27*01': 0.273},
 'cccagccacaaacaagtatttttaagcaaaaga': {'IG

In [95]:
top_matches_d = []

for gene,matches in partials_d.items():
    for allele,perc in matches.items():
        if perc >= 0.35:
            top_matches_d.append( (allele,perc,gene) )

top_matches_d

[('IGHD7-27*01', 0.364, 'tattactgtgtgagagg'),
 ('IGHD7-27*01', 0.364, 'gggaatctagg'),
 ('IGHD1/OR15-1a*01', 0.353, 'gtgcgggaacagca'),
 ('IGHD1/OR15-1b*01', 0.353, 'gtgcgggaacagca'),
 ('IGHD7-27*01', 0.364, 'gtgcgggaacagca'),
 ('IGHD7-27*01', 0.364, 'ctaagaaaatatctt'),
 ('IGHD7-27*01', 0.364, 'gggactctaac')]

##### J genes

In [89]:
    # -------------------- SEARCH FOR J GENES --------------------
jgenes = []
printed = 0

# Read fasta sequence
for nt in SeqIO.parse(locus_file, "fasta"):
     nts = str(nt.seq).lower()

    # Get nt sequences in three reading frames
    nt_len = len(nt.seq)
    nt_frame = ['','','']
    nt_frame[0] = nt[0 : nt_len - (nt_len + 0)%3]
    nt_frame[1] = nt[1 : nt_len - (nt_len + 2)%3]
    nt_frame[2] = nt[2 : nt_len - (nt_len + 1)%3]

    # Get aa translations in three reading frames
    aa_frame = ['','','']
    aa_frame[0] = str(nt_frame[0].seq.translate())
    aa_frame[1] = str(nt_frame[1].seq.translate())
    aa_frame[2] = str(nt_frame[2].seq.translate())

    # Uncomment to skip search for V and J genes
    #aa_frame[0] = ''; aa_frame[1] = ''; aa_frame[2] = ''
    
    
    # Search for W - 8 residues - SS
    # Then search backwards to conserved flanking heptamer

    # Uncomment to skip search for V and J genes
    # aa_frame[0] = ''; aa_frame[1] = ''; aa_frame[2] = ''

    for frame,offset in zip(aa_frame,[0,1,2]):
        for mcons in finditer('W[A-Z]{8}SS', frame):
            sta_aa = mcons.span()[0]         # Starting aa in frame
            end_aa = mcons.span()[1]         # End aa in frame
            sta_nt = sta_aa*3 + offset       # Start nt in original seq
            end_nt = end_aa*3 + offset       # End nt in original sequence
            
            # Search upstream for heptamer
            found     = False
            for i in range(0,39):
                # upstream heptamer consensus = cactgtg
                upstream_heptamer = nts[sta_nt-i-7:sta_nt-i]
                upstream_heptamer_score = 0
                if upstream_heptamer[0] == 'c': upstream_heptamer_score += 1
                if upstream_heptamer[1] == 'a': upstream_heptamer_score += 1
                if upstream_heptamer[2] == 'c': upstream_heptamer_score += 1
                if upstream_heptamer[3] == 't': upstream_heptamer_score += 1

                # upstream nonamer consensus = ggtttttgt
                upstream_nonamer = nts[sta_nt-i-7-23-9:sta_nt-i-7-23]
                upstream_nonamer_score = 0
                if upstream_nonamer[0] == 'g': upstream_nonamer_score += 1
                if upstream_nonamer[1] == 'g': upstream_nonamer_score += 1
                if upstream_nonamer[2] == 't': upstream_nonamer_score += 1
                if upstream_nonamer[3] == 't': upstream_nonamer_score += 1
                if upstream_nonamer[4] == 't': upstream_nonamer_score += 1
                if upstream_nonamer[5] == 't': upstream_nonamer_score += 1
                if upstream_nonamer[6] == 't': upstream_nonamer_score += 1
                if upstream_nonamer[7] == 'g': upstream_nonamer_score += 1
                if upstream_nonamer[8] == 't': upstream_nonamer_score += 1


                # Setting nonamer_score lower than 6 leads to more false positives and breaks results
                if (upstream_heptamer[4:7] == 'gtg' and upstream_heptamer_score >= 2 
                    and upstream_nonamer_score >= 5):
                    sta_nt -= i
                    found = True
                    break


            gene     = nts[sta_nt:end_nt]
            gene_len = end_nt - sta_nt

            # Temporary collection of genes identified by script
            jgenes.append(gene)
            
            # Look for exact matches between discovered gene and known alleles
            # Allow for the possibility that multiple alleles have same sequence
            # Default assumption is that gene is not in database

            allele = 'Not in J ref db'
            for jallele, jseq in jseq_dict.items():
                if gene == jseq:
                    if allele == 'Not in J ref db':
                        allele = jallele + ' ' + jtype_dict[jallele]
                    else:
                        allele = allele + ', ' + jallele + ' ' + jtype_dict[jallele]

            all_returns[gene] = allele
            
            # Flag J genes that start before last V gene
            if sta_nt < last_v_nt:
                continue
                # annot += ' / located in V gene region'

            if found:
                printed_j += 1
                print(f">{allele} {gene_len} nts: {sta_nt} - {end_nt}\n")
                print(gene)
                print("\n")

>Not in J ref db 52 nts: 1014593 - 1014645

ctactggtacttcgatctctggggccgtggcaccctggtcactgtctcctca


>Not in J ref db 49 nts: 1015208 - 1015257

tgatgcttttgatatctggggccaagggacaatggtcaccgtctcttca


>Not in J ref db 47 nts: 1015582 - 1015629

actactttgactactggggccagggaaccctggtcaccgtctcctca


>IGHJ6*03 F 62 nts: 1016584 - 1016646

attactactactactactacatggacgtctggggcaaagggaccacggtcaccgtctcctca


>Not in J ref db 51 nts: 1014386 - 1014437

gctgaatacttccagcactggggccagggcaccctggtcaccgtctcctca


>Not in J ref db 50 nts: 1015980 - 1016030

acaactggttcgacccctggggccagggaaccctggtcaccgtctcctca




In [90]:
not_found = [ gene for gene,allele in all_returns.items() if allele == 'Not in J ref db']
len(not_found)

157

In [91]:
partials_j = {}

for gene in not_found:
    partials_j[gene] = compare_partials( gene,jseq_dict )

partials_j

{'acaactggttcgacccctggggccagggaaccctggtcaccgtctcctca': {'IGHJ1*01': 0.385,
  'IGHJ2*01': 0.208,
  'IGHJ3*01': 0.26,
  'IGHJ3*02': 0.26,
  'IGHJ4*01': 0.479,
  'IGHJ4*02': 0.688,
  'IGHJ4*03': 0.417,
  'IGHJ5*01': 0.451,
  'IGHJ5*02': 0.961,
  'IGHJ6*01': 0.238,
  'IGHJ6*02': 0.242,
  'IGHJ6*03': 0.242,
  'IGHJ6*04': 0.238},
 'actactttgactactggggccagggaaccctggtcaccgtctcctca': {'IGHJ1*01': 0.385,
  'IGHJ2*01': 0.208,
  'IGHJ3*01': 0.26,
  'IGHJ3*02': 0.26,
  'IGHJ4*01': 0.479,
  'IGHJ4*02': 0.958,
  'IGHJ4*03': 0.438,
  'IGHJ5*01': 0.451,
  'IGHJ5*02': 0.647,
  'IGHJ6*01': 0.238,
  'IGHJ6*02': 0.242,
  'IGHJ6*03': 0.242,
  'IGHJ6*04': 0.238},
 'ctactggtacttcgatctctggggccgtggcaccctggtcactgtctcctca': {'IGHJ1*01': 0.269,
  'IGHJ2*01': 0.962,
  'IGHJ3*01': 0.18,
  'IGHJ3*02': 0.18,
  'IGHJ4*01': 0.229,
  'IGHJ4*02': 0.229,
  'IGHJ4*03': 0.229,
  'IGHJ5*01': 0.216,
  'IGHJ5*02': 0.216,
  'IGHJ6*01': 0.127,
  'IGHJ6*02': 0.145,
  'IGHJ6*03': 0.129,
  'IGHJ6*04': 0.127},
 'gctgaatacttccagcactgg

In [96]:
top_matches_j = []

for gene,matches in partials_j.items():
    for allele,perc in matches.items():
        if perc >= 0.75:
            top_matches_j.append( (allele,perc,gene) )

top_matches_j

[('IGHJ2*01', 0.962, 'ctactggtacttcgatctctggggccgtggcaccctggtcactgtctcctca'),
 ('IGHJ3*02', 0.96, 'tgatgcttttgatatctggggccaagggacaatggtcaccgtctcttca'),
 ('IGHJ4*02', 0.958, 'actactttgactactggggccagggaaccctggtcaccgtctcctca'),
 ('IGHJ1*01', 0.962, 'gctgaatacttccagcactggggccagggcaccctggtcaccgtctcctca'),
 ('IGHJ5*02', 0.961, 'acaactggttcgacccctggggccagggaaccctggtcaccgtctcctca')]